# Basic usage

Welcome to the basic usage notebook of pySWAP package. Here we will go through the basics of how a SWAP model is constructed in pySWAP, load one of the test case models and explore it a bit.

In [1]:
import pyswap as psp

psp.log.set_log_level("WARNING")

pySWAP is an object-oriented library, which means that we try to split and encapsulate distinct parts of the SWAP model setup as objects. These objects need to be defined by the user at the initial stage and then can later be easily manipulated, copied and compared. Below is an example of the Meteorological settings object:

## Hupselbrook testcase

A lot of the model settings still need to be explicitly written out. There is, however, an ongoing effort to reduce to minimum the amount of code that has to be written for the initial model. If you are interested how the Hupselbrook model is implemented in pySWAP, take a look at file `hupselbrook.py` in `testcase` module.

Let's load the model and check the metadata.

In [2]:
ml = psp.testcase.get("hupselbrook")

AttributeError: module 'pyswap.components.crop' has no attribute 'CHTB'

`Model` object has a method run, which runs the model and captures the results into a `Result` object. The only argument that needs to be passed is the path to where the temporary directory can be created.

In [ ]:
print(ml.swp)

In [ ]:
result = ml.run("./")

From now on, we can use Result object's attributes and computed fields to access the results. The .csv, for example, is already loaded as DataFrame.

In [ ]:
result.output.keys()

In [ ]:
result.yearly_summary

In [ ]:
print(result.blc_summary)

In [ ]:
result.csv[["RAIN", "IRRIG", "INTERC"]].plot()

## Alternative way to load a model

Since pyswap 0.2.0, it is possible to load an entire model from the classic ascii configuration files. In order to do that, you need to create the swp and other relevant files separately and then put them together in a model.

In [ ]:

meta = Metadata(
    author="John Doe",
    institution="University of Somewhere",
    email="john.doe@somewhere.com",
    project="pySWAP test - hupselbrook",
    swap_ver="4.2",
)

ml = load_swp(hupsel_swp_path, meta)


hupsel_dra_path = testcase.load_dataset.RESOURCES["hupselbrook"]["dra"]

dra = load_dra(hupsel_dra_path)


## Changing the model settings

Assume you would like to add another file extension to be generated, or make any other change for that matter. The best way to achieve this is to create a copy of a component you want to alter, and then update the model with that new component, saving it's copy.

In [ ]:
# updated general settings including inlist_csv_tz
generalsettings_csv_tz = ml.generalsettings.model_copy(deep=True)

In [ ]:
print(generalsettings_csv_tz.model_string())

In [ ]:
generalsettings_csv_tz.add_extension("csv_tz", inlist=["WC", "H"])

In [ ]:
print(generalsettings_csv_tz.model_string())

In [ ]:
ml2 = ml.model_copy(update={"generalsettings": generalsettings_csv_tz})

In [ ]:
print(ml2.swp)

In [ ]:
result2 = ml2.run("./")

In [ ]:
result2.output.keys()

In [ ]:
result.csv_tz.head()  # this will be None
result2.csv_tz.head()  # this will show a DataFrame with the WC and H columns

In [ ]:
print(result2.csv_tz)

In [ ]:
from pyswap.core.plot import water_content

wc_df = result2.csv_tz.reset_index()

water_content(wc_df, "DEPTH", "DATE", "WC")

## Alternative way to setup the model

Initially it was only possible to directly assign generated model components to the Model object attributes. Since the newest release (November 2024) it is possible to set up and run the model by adding the individual 